In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [16]:
pd.DataFrame(json.loads(json_data))

,feature1,feature2
0,1,4
1,2,5
2,3,6


In [6]:
import json

In [15]:
df = pd.DataFrame({'feature1': [1, 2, 3], 'feature2': [4, 5, 6]})

# Convert DataFrame to JSON string
json_data = df.to_json(orient='records')
json_data

'[{"feature1":1,"feature2":4},{"feature1":2,"feature2":5},{"feature1":3,"feature2":6}]'

In [17]:
from io import StringIO 
import boto3
import sagemaker
from sagemaker.session import s3_input, Session
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sagemaker.amazon.amazon_estimator import image_uris,get_image_uri
from sagemaker.serializers import CSVSerializer

role = get_execution_role()
sess = Session()
region = sess.boto_region_name
bucket= sess.default_bucket()
prefix = 'titanic'

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [48]:
s3 = boto3.client('s3',region_name=region)
# obj = s3.get_object(Bucket=bucket,Key=f'{prefix}/input/train.csv')
# train = pd.read_csv(obj['Body'])

# obj = s3.get_object(Bucket=bucket,Key=f'{prefix}/input/test.csv')
# test = pd.read_csv(obj['Body'])

# upload_data = sess.upload_data('data',key_prefix='titanic/input')

In [52]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [53]:
train.head()

,Survived,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [54]:
train.isnull().sum()

Survived         0
PassengerId      0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [55]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Survived     891 non-null    int64  
 1   PassengerId  891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## Model

In [86]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from container.decision_trees.ml import custom as ml_custom
from container.decision_trees.ml import fillna as ml_fillna
import category_encoders as ce


def pipe():

    model = Pipeline(steps=[
        ('drop',ml_custom.ExtractColumn(columns=['Pclass', 'Sex', 'Age', 'SibSp',
           'Parch',  'Fare', 'Cabin', 'Embarked'])),
        ('na',ml_fillna.FillMissingValue(cat_cols=['Cabin','Embarked'],num_cols=['Age'])),
        ('ohe',ce.OneHotEncoder(cols=['Sex','Cabin','Embarked'])),
        ('model',DecisionTreeClassifier(max_depth=10))
    ])
    return model

In [83]:
X = train.iloc[:,1:]
y = train.iloc[:,0]

In [84]:
model.fit(X,y)

/home/ec2-user/SageMaker/titanic-sagemaker/container/decision_trees/ml/fillna.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].fillna(mode.values[0][0])
/home/ec2-user/SageMaker/titanic-sagemaker/container/decision_trees/ml/fillna.py:87: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  X[col] = X[col].fillna(mean[0])
/home/ec2-user/SageMaker/titanic-sagemaker/container/decision_trees/ml/fillna.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Pipeline(steps=[('drop',
                 ExtractColumn(columns=['Pclass', 'Sex', 'Age', 'SibSp',
                                        'Parch', 'Fare', 'Cabin',
                                        'Embarked'])),
                ('na',
                 FillMissingValue(cat_cols=['Cabin', 'Embarked'],
                                  num_cols=['Age'])),
                ('ohe', OneHotEncoder(cols=['Sex', 'Cabin', 'Embarked'])),
                ('model', DecisionTreeClassifier(max_depth=10))])